# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from the .env file
load_dotenv(find_dotenv())

# Access the API key using os.getenv
openai_api_key = os.getenv("OPENAI_API_KEY")

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_api_key,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010. He drove for Red Bull Racing and secured his first of four consecutive championships that year.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

The 2011 F1 World Championship was won by Sebastian Vettel of Germany driving for Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [5]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [6]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [7]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [8]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [9]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.  
Team: Renault.


Few Shots for classification.


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [11]:
# Define the context with few-shot examples
context_user = [
    {'role':'system', 'content':"""You are an expert in F1.

    Who won the 2010 F1 championship?
    Driver: Sebastian Vettel.
    Team: Red Bull Racing.
    Points: 256.

    Who won the 2009 F1 championship?
    Driver: Jenson Button.
    Team: BrawnGP.
    Points: 95.

    Who won the 2006 F1 championship?
    Driver: Fernando Alonso.
    Team: Renault.
    Points: 134.

    Who won the 2000 F1 championship?
    Driver: Michael Schumacher.
    Team: Ferrari.
    Points: 108.
    """}
]

# Query for the 1994 F1 championship winner
response_1994 = return_OAIResponse("Who won the 1994 F1 championship?", context_user)
print(response_1994)

# Query for the 1988 F1 championship winner
response_1988 = return_OAIResponse("Who won the 1988 F1 championship?", context_user)
print(response_1988)

# Query for the 1992 F1 championship winner
response_1992 = return_OAIResponse("Who won the 1992 F1 championship?", context_user)
print(response_1992)

Driver: Michael Schumacher.
Team: Benetton.
Points: 92.
Driver: Ayrton Senna.
Team: McLaren-Honda.
Points: 94.
Driver: Nigel Mansell.
Team: Williams.
Points: 108.


In [12]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


In [13]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I bought this coffee maker a month ago, and it's been a game-changer for my mornings. Easy to use and makes great coffee. Highly recommend!", context_user))


Sentiment: Positive


In [14]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("After the latest update, the app crashes every time I try to open it. Really frustrating, as I rely on this app daily. Please fix it soon!", context_user))

Sentiment: Negative


In [15]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("The LED lamp is nice, but it's a bit smaller than I expected. It does the job, but I might look for a larger one next time.", context_user))

Sentiment: Neutral


# **Report on M-Shots Learning with GPT-3.5**

This report investigates the use of M-Shots learning techniques with GPT-3.5, focusing on guiding the model towards desired responses through prompts. We analyze both successes and challenges encountered during experimentation.

## **Methodology**

We employed M-Shots learning, including zero-shot, one-shot, and few-shot approaches, with GPT-3.5. We provided the model with prompts designed to elicit responses in specific formats, analyzing its output for alignment and accuracy.

## **Findings**

### **Challenges:**

* **GPT-3.5 Hallucinations:** In some cases, the model generated irrelevant or factually incorrect information, particularly for unfamiliar contexts or ambiguous prompts.
* **Input Overload:** Utilizing a large number of examples (over six) overwhelmed the model, hindering its performance.

### **Key Learnings:**

* **M-Shots Effectiveness:** M-Shots learning demonstrated success in shaping GPT-3.5 responses. The model adapted its output based on structured prompts and examples.
* **Example Optimization:** One or few shots proved optimal for scenarios requiring precise formatting or detailed responses. This approach prevented overwhelming the model with excessive information.
* **Role Importance:** Defining roles (system, user, assistant) within prompts enhanced understanding and accuracy. This facilitated a more natural conversation and iterative learning.
* **Prompt Design Best Practices:** Clear instructions, specified formats, and relevant examples within prompts improved learning and ensured consistent, accurate responses.

## **Conclusion**

M-Shots learning presents a valuable framework for guiding AI responses. By effectively leveraging structured prompts and examples, we can significantly enhance GPT-3.5's performance in generating accurate and relevant outputs. However, careful consideration of input limitations, role definitions, and prompt clarity is crucial to ensure success. This approach contributes to the ongoing advancement of AI language models, paving the way for more sophisticated and contextually aware interactions.

# **What i have learned:**

Through the exploration of M-Shots learning techniques with GPT-3.5:

1. **Effective Prompt Design**: Structuring prompts with clear instructions and examples is crucial for guiding the model to produce accurate and relevant responses.
  
2. **Role of Context**: Incorporating roles (system, user, assistant) in prompts enhances the model's understanding and improves response accuracy by simulating natural conversation flows.

3. **Limitations and Challenges**: The model may struggle with unfamiliar or ambiguous contexts, leading to inaccuracies or irrelevant responses. Managing input complexity and ensuring concise prompts are key to mitigating these challenges.

4. **Optimal Use of Examples**: Using one or few shots is generally effective for teaching the model specific response formats, balancing learning efficiency with input capacity constraints.

5. **Continuous Improvement**: There's a need for ongoing refinement in prompt design and context management to maximize the model's capabilities and reliability in real-world applications.

Overall, M-Shots learning provides a structured approach to harnessing GPT-3.5's capabilities for tailored responses, highlighting both its strengths and areas for continued development and optimization.